In [1]:
import numpy as np
import os
import pandas as pd #data handling library
import sklearn as sk

def get_RPKM_by_file(file_name, data_fields):
    df = pd.read_table(file_name, usecols=data_fields)
    df = df.T # tanspose
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df = df.rename(columns = new_header.T) #set the header row as the df header
    return df

data_dir = './Data'
data_fields = ('gene','RPKM')
RPKM_list = []
for dirName, subdirList, fileList in os.walk(data_dir):
    RPKM_list.extend(map(lambda f: get_RPKM_by_file(os.path.join(dirName, f), data_fields), fileList))

RPKM_list

[     ?|100130426 ?|100133144 ?|100134869   ?|10357   ?|10431 ?|136542  \
 RPKM  0.06694772   0.9270708   0.4969419  5.518766  22.36725        0   
 
       ?|155060    ?|26823 ?|280660 ?|317712    ?|340602     ?|388795  \
 RPKM  2.880182  0.2761594        0        0  0.01408699  0.006895903   
 
       ?|390284    ?|391343    ?|391714 ?|404770 ?|441362 ?|442388 ?|553137  \
 RPKM  0.247565  0.01040163  0.01941341        0        0        0        0   
 
        ?|57714      
 RPKM  4.278614 ...  
 
 [1 rows x 20532 columns],
      ?|100130426 ?|100133144 ?|100134869   ?|10357   ?|10431 ?|136542  \
 RPKM           0    1.180236    0.450388  4.992169  24.97131        0   
 
       ?|155060    ?|26823 ?|280660 ?|317712 ?|340602   ?|388795   ?|390284  \
 RPKM  8.243926  0.2788105        0        0        0  0.0357388  0.2138389   
 
      ?|391343 ?|391714 ?|404770 ?|441362 ?|442388    ?|553137  ?|57714      
 RPKM        0        0        0        0        0  0.08234143  8.80007 ...  
 
 

In [2]:

#def get_RPKM_by_file(files):
#    for file_name in files:
#        df = pd.read_table(file_name, usecols=data_fields)
#        df = df.T # tanspose
#        new_header = df.iloc[0] #grab the first row for the header
#        df = df[1:] #take the data less the header row
#        df = df.rename(columns = new_header.T) #set the header row as the df header
#        RPKM_list.append(df)
#    return RPKM_list



#flattened_data = get_RPKM_by_file(file_list)




In [3]:
def merge_data(data_list):
    df = pd.concat(data_list, axis=0, join='inner', join_axes=None, ignore_index=False,
        keys=None, levels=None, names=None, verify_integrity=False)
    return df

X = merge_data(RPKM_list)
Y = np.ravel(map(round, np.random.rand(len(X), 1)))

print X

     ?|100130426 ?|100133144 ?|100134869   ?|10357   ?|10431 ?|136542  \
RPKM  0.06694772   0.9270708   0.4969419  5.518766  22.36725        0   
RPKM           0    1.180236    0.450388  4.992169  24.97131        0   
RPKM  0.04956242   0.9206793   0.3488647  3.163578  14.72296        0   
RPKM           0     1.09992   0.4067801  3.894188  22.01988        0   
RPKM           0   0.3333464  0.07578716  3.343024  25.94024        0   
RPKM           0    1.221118   0.4479714  5.909189  31.01896        0   

       ?|155060    ?|26823 ?|280660 ?|317712     ?|340602     ?|388795  \
RPKM   2.880182  0.2761594        0        0   0.01408699  0.006895903   
RPKM   8.243926  0.2788105        0        0            0    0.0357388   
RPKM   6.713778  0.4088899        0        0            0   0.09444512   
RPKM   2.999885  0.2930964        0        0            0  0.005367145   
RPKM  0.9158181  0.2516765        0        0  0.008307013            0   
RPKM   6.183879  0.1899848        0        0

In [4]:
# Impute missing data, assuming X is all numeric values. Y can be strings.
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X = imp.fit_transform(X)

imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
Y = imp.fit_transform(Y)

In [5]:
# Encode classes as numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
nonnumeric_columns_X = []
nonnumeric_columns_Y = []
if nonnumeric_columns_X:
    for feature in nonnumeric_columns_X:
        X[feature] = le.fit_transform(X[feature])
if nonnumeric_columns_Y:
    for feature in nonnumeric_columns_Y:
        Y[feature] = le.fit_transform(Y[feature])

In [6]:
# Split data into train and test sets.
def split_data(X, Y, p_train):
    msk = np.random.rand(len(RPKM_list)) < p_train
    print Y[0]
    print msk
    X_train = X[msk]
    Y_train = Y[0][msk]
    return X[msk], X[~msk], Y[0][msk], Y[0][~msk]

train_X, test_X, train_Y, test_Y = split_data(X, Y, 0.7)
print train_Y
print test_Y

[ 1.  0.  1.  1.  1.  1.]
[ True  True  True  True  True  True]
[ 1.  0.  1.  1.  1.  1.]
[]


In [8]:
from utility import statsTest
stats = statsTest.statistics_test(pd.DataFrame(train_X), pd.DataFrame(train_Y))
print stats[0:10]

['pvalue']


In [2]:
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_Y)
predictions = gbm.predict(test_X)
predictions

NameError: name 'train_X' is not defined

ImportError: No module named StatsTest